In [2]:
import os
import sys

# Add project root to sys.path so you can import src modules
project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
%load_ext autoreload
%autoreload 2
import importlib